# Manejando la data de Netflix

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

In [2]:
# importar el csv descargado en nuestra computadora

netflix = pd.read_csv("../../data/netflix_titles.csv")
netflix.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."


## Adding new columns

In [3]:
# Total directors
netflix[ 'director'].str.split( ',' , expand = True ).notnull()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7782,True,False,False,False,False,False,False,False,False,False,False,False,False
7783,True,False,False,False,False,False,False,False,False,False,False,False,False
7784,False,False,False,False,False,False,False,False,False,False,False,False,False
7785,False,False,False,False,False,False,False,False,False,False,False,False,False


In [4]:
netflix[ 'num_director'] = netflix[ 'director'].str.split( ',' , expand = True ).notnull().sum( axis = 1 )

In [5]:
# Total countries

netflix[ 'country' ]

0                                                  Brazil
1                                                  Mexico
2                                               Singapore
3                                           United States
4                                           United States
                              ...                        
7782    Sweden, Czech Republic, United Kingdom, Denmar...
7783                                                India
7784                                                  NaN
7785                                            Australia
7786                United Kingdom, Canada, United States
Name: country, Length: 7787, dtype: object

In [6]:
netflix[ 'country'].str.split( ',' , expand = True )

,0,1,2,3,4,5,6,7,8,9,10,11
0,Brazil,None,None,None,None,None,None,None,None,None,None,None
1,Mexico,None,None,None,None,None,None,None,None,None,None,None
2,Singapore,None,None,None,None,None,None,None,None,None,None,None
3,United States,None,None,None,None,None,None,None,None,None,None,None
4,United States,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,Sweden,Czech Republic,United Kingdom,Denmark,Netherlands,None,None,None,None,None,None,None
7783,India,None,None,None,None,None,None,None,None,None,None,None
7784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7785,Australia,None,None,None,None,None,None,None,None,None,None,None


In [7]:
netflix[ 'num_ctry'] = netflix[ 'country'].str.split( ',' , expand = True ).notnull().sum( axis = 1 )

## filtering NA directors and countries

In [8]:
netflix.loc[ netflix['num_ctry'] == 0, 'num_ctry'] = None

In [9]:
netflix.loc[ netflix['num_director'] == 0, 'num_director'] = None

## Groupby

In [10]:
# Differents type
netflix[ 'type' ].unique()

array(['TV Show', 'Movie'], dtype=object)

In [11]:
netflix.groupby( [ 'type' ] ).mean()

,release_year,num_director,num_ctry
type,,,
Movie,2012.920030,1.127158,1.284826
TV Show,2016.191701,1.293478,1.150492


## Extra examples of group by

Inspiration from this [code](https://stackoverflow.com/questions/19384532/get-statistics-for-each-group-such-as-count-mean-etc-using-pandas-groupby).

In [12]:
# Not null values
netflix.groupby( [ 'type' ] )[['num_ctry', 'num_director']].count()

,num_ctry,num_director
type,,
Movie,5147,5214
TV Show,2133,184


In [ ]:
netflix.groupby( [ 'type' ] )[['num_ctry', 'num_director']].max().reset_index( level = [ 'type'] )

## diferent statistics

In [ ]:
netflix.groupby( [ 'type' ] )[['num_ctry', 'num_director']].agg( ['min', 'max'] )

In [ ]:
netflix.groupby( [ 'type' ] )[['num_ctry', 'num_director']].agg( num_ctry_min = pd.NamedAgg( column = "num_ctry", aggfunc = "min" ), num_direc_max = pd.NamedAgg( column = "num_director", aggfunc = "max" ) )

## Merge 

In [ ]:
netflix_stats = netflix.groupby( [ 'type' ] )[['num_ctry', 'num_director']].agg( num_ctry_min = pd.NamedAgg( column = "num_ctry", aggfunc = "min" ), num_direc_max = pd.NamedAgg( column = "num_director", aggfunc = "max" ) ).reset_index( level = ['type'] )

In [ ]:
netflix.merge( netflix_stats , how = 'right', on = 'type' , indicator = True )